# Analysis of 2019 Sales

## Data Wrangling & Cleansing

### Setup

In [146]:
# import numpy as np
import pandas as pd
# import seaborn as sns

%matplotlib inline
%reload_ext autoreload
%autoreload 2

### Collate source data

In [147]:
# import os

# df = pd.DataFrame()

# for file in os.listdir('./sales_data'):
#     month = pd.read_csv(f'./sales_data/{ file }', skip_blank_lines=True)
#     df = pd.concat([df, month], ignore_index=True, copy=False)
#     print(f'{file} { month.shape } => { df.shape }')

# df.tail()

### Remove superfluous header rows (interleaved in the data)

In [148]:
# df[df['Order ID'] == 'Order ID'].describe()

Attempting to drop headers via `read_csv()` above added columns. And as
there's `355` header rows within the data, I'm removing altogether here

In [149]:
# # Drop header rows
# df = df[df['Order ID'] != 'Order ID']

# df[df['Order ID'] == 'Order ID'].describe()

### Remove superfluous NaN rows

In [150]:
# df[df.isna().any(axis=1)].info()
# df[df.isna().any(axis=1)].tail()

`545` rows containing `NaN` (or equivalent) in `any` column

In [151]:
# # Drop NaN rows
# df = df[~df.isna().any(axis=1)]

# df[df.isna().any(axis=1)].info()

### Persist combined dataframe

In [152]:
# df.to_csv('2019-sales.cleaned.csv', index=False)

### Reload dataframe `df` from combined file

... with the added benefit that Pandas correctly identifies most column data types

In [153]:
df = pd.read_csv('2019-sales.cleaned.csv')

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 185950 entries, 0 to 185949
Data columns (total 6 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Order ID          185950 non-null  int64  
 1   Product           185950 non-null  object 
 2   Quantity Ordered  185950 non-null  int64  
 3   Price Each        185950 non-null  float64
 4   Order Date        185950 non-null  object 
 5   Purchase Address  185950 non-null  object 
dtypes: float64(1), int64(2), object(3)
memory usage: 8.5+ MB


### Order Date => `datetime`

In [154]:
df['Order Date']

0         04/19/19 08:46
1         04/07/19 22:30
2         04/12/19 14:38
3         04/12/19 14:38
4         04/30/19 09:27
               ...      
185945    09/17/19 20:56
185946    09/01/19 16:00
185947    09/23/19 07:39
185948    09/19/19 17:30
185949    09/30/19 00:18
Name: Order Date, Length: 185950, dtype: object

In [155]:
df['Order Date'] = pd.to_datetime(df['Order Date'], format='%m/%d/%y %H:%M')
df['Order Date']

0        2019-04-19 08:46:00
1        2019-04-07 22:30:00
2        2019-04-12 14:38:00
3        2019-04-12 14:38:00
4        2019-04-30 09:27:00
                 ...        
185945   2019-09-17 20:56:00
185946   2019-09-01 16:00:00
185947   2019-09-23 07:39:00
185948   2019-09-19 17:30:00
185949   2019-09-30 00:18:00
Name: Order Date, Length: 185950, dtype: datetime64[ns]

In [156]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 185950 entries, 0 to 185949
Data columns (total 6 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   Order ID          185950 non-null  int64         
 1   Product           185950 non-null  object        
 2   Quantity Ordered  185950 non-null  int64         
 3   Price Each        185950 non-null  float64       
 4   Order Date        185950 non-null  datetime64[ns]
 5   Purchase Address  185950 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(2), object(2)
memory usage: 8.5+ MB


## Data Elaboration

In [157]:
df.head()

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
0,176558,USB-C Charging Cable,2,11.95,2019-04-19 08:46:00,"917 1st St, Dallas, TX 75001"
1,176559,Bose SoundSport Headphones,1,99.99,2019-04-07 22:30:00,"682 Chestnut St, Boston, MA 02215"
2,176560,Google Phone,1,600.00,2019-04-12 14:38:00,"669 Spruce St, Los Angeles, CA 90001"
3,176560,Wired Headphones,1,11.99,2019-04-12 14:38:00,"669 Spruce St, Los Angeles, CA 90001"
4,176561,Wired Headphones,1,11.99,2019-04-30 09:27:00,"333 8th St, Los Angeles, CA 90001"


### Line Total

In [158]:
df['Line Total'] = df['Quantity Ordered'] * df['Price Each']

### Order Date

In [159]:
import calendar

# months = [calendar.month(2019, 1 + month) for month in range(12)]
# print(''.join(months))

months = [calendar.monthcalendar(2019, 1 + month) for month in range(12)]

def week_of_month(month = 1, day = 1):
    # print(months[month - 1])
    for idx, week in enumerate(months[month - 1]):
        if day in week:
            return idx + 1


df['At_Quarter_Change'] = (df['Order Date'].dt.is_quarter_start | df['Order Date'].dt.is_quarter_end)
df['Quarter'] = df['Order Date'].dt.quarter
df['At_Month_Change'] = (df['Order Date'].dt.is_month_start | df['Order Date'].dt.is_month_end)
df['ordinal_Month'] = df['Order Date'].dt.month
df['Month'] = df['Order Date'].dt.month_name()
df['ordinal_Day'] = df['Order Date'].dt.day
df['Day'] = df['Order Date'].dt.day_name()
df['Week_Of_Month'] = df['Order Date'].apply(lambda r: week_of_month(r.month, r.day))

df.head()

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address,Line Total,At_Quarter_Change,Quarter,At_Month_Change,ordinal_Month,Month,ordinal_Day,Day,Week_Of_Month
0,176558,USB-C Charging Cable,2,11.95,2019-04-19 08:46:00,"917 1st St, Dallas, TX 75001",23.90,False,2,False,4,April,19,Friday,3
1,176559,Bose SoundSport Headphones,1,99.99,2019-04-07 22:30:00,"682 Chestnut St, Boston, MA 02215",99.99,False,2,False,4,April,7,Sunday,1
2,176560,Google Phone,1,600.00,2019-04-12 14:38:00,"669 Spruce St, Los Angeles, CA 90001",600.00,False,2,False,4,April,12,Friday,2
3,176560,Wired Headphones,1,11.99,2019-04-12 14:38:00,"669 Spruce St, Los Angeles, CA 90001",11.99,False,2,False,4,April,12,Friday,2
4,176561,Wired Headphones,1,11.99,2019-04-30 09:27:00,"333 8th St, Los Angeles, CA 90001",11.99,False,2,True,4,April,30,Tuesday,5


### Address

In [160]:
# FIXME Takes 1 - 5min!!! to execute

# def parse_address(address = ""):
#     city = ','.join(address.split(',')[1:])
#     city = (city[:-6]).strip()
#     return {
#         'city': city,
#         'state': city[-2:],
#     }

# https://stackoverflow.com/questions/39050539/how-to-add-multiple-columns-to-pandas-dataframe-in-one-assignment
# Append multiple columns when parsing the `Purchase Address` column into `city` and `state`
# df = df.join(
#     df['Purchase Address'].apply(lambda r: pd.Series(parse_address(r)))
# )

In [161]:
# Takes 1.7 - 4.2 seconds!!
# Step 1: Split the Address into Street and rest
# `840 Highland St, Los Angeles, CA 90001` => ['840 Highland St', 'Los Angeles, CA 90001']
df[['Street','City_State_Zip']] = df['Purchase Address'].str.split(', ', n=1, expand=True)

# Step 2: Split the `[1]` (the rest) index above into City, State and Zip
# `Los Angeles, CA 90001` => ['Los Angeles, CA','90001']
df[['City','Zip']] = df['City_State_Zip'].str.rsplit(' ', n=1, expand=True)

# Step 3: Split the `[0]` (City) index above into city-only and State (`[1]`)
# `Los Angeles, CA` => [ _ , 'CA']
df['State'] = df['City'].str.rsplit(', ', expand=True)[1]

# Step 4: Drop superfluous Purchase Address and WIP columns
df.drop([
    'Purchase Address', 
    'Street', 
    'City_State_Zip', 
    'Zip'],
    axis=1,
    inplace=True)

df.tail()

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Line Total,At_Quarter_Change,Quarter,At_Month_Change,ordinal_Month,Month,ordinal_Day,Day,Week_Of_Month,City,State
185945,259353,AAA Batteries (4-pack),3,2.99,2019-09-17 20:56:00,8.97,False,3,False,9,September,17,Tuesday,4,"Los Angeles, CA",CA
185946,259354,iPhone,1,700.00,2019-09-01 16:00:00,700.00,False,3,True,9,September,1,Sunday,1,"San Francisco, CA",CA
185947,259355,iPhone,1,700.00,2019-09-23 07:39:00,700.00,False,3,False,9,September,23,Monday,5,"San Francisco, CA",CA
185948,259356,34in Ultrawide Monitor,1,379.99,2019-09-19 17:30:00,379.99,False,3,False,9,September,19,Thursday,4,"San Francisco, CA",CA
185949,259357,USB-C Charging Cable,1,11.95,2019-09-30 00:18:00,11.95,True,3,True,9,September,30,Monday,6,"San Francisco, CA",CA


### Categorical Features

In [162]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 185950 entries, 0 to 185949
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   Order ID           185950 non-null  int64         
 1   Product            185950 non-null  object        
 2   Quantity Ordered   185950 non-null  int64         
 3   Price Each         185950 non-null  float64       
 4   Order Date         185950 non-null  datetime64[ns]
 5   Line Total         185950 non-null  float64       
 6   At_Quarter_Change  185950 non-null  bool          
 7   Quarter            185950 non-null  int32         
 8   At_Month_Change    185950 non-null  bool          
 9   ordinal_Month      185950 non-null  int32         
 10  Month              185950 non-null  object        
 11  ordinal_Day        185950 non-null  int32         
 12  Day                185950 non-null  object        
 13  Week_Of_Month      185950 non-null  int64   

#### Product

In [163]:
factorised_Product = pd.factorize(df['Product'])
df['ordinal_Product'] = factorised_Product[0]

df[['ordinal_Product', 'Product']]

,ordinal_Product,Product
0,0,USB-C Charging Cable
1,1,Bose SoundSport Headphones
2,2,Google Phone
3,3,Wired Headphones
4,3,Wired Headphones
...,...,...
185945,9,AAA Batteries (4-pack)
185946,10,iPhone
185947,10,iPhone
185948,18,34in Ultrawide Monitor


#### City

In [164]:
factorised_City = pd.factorize(df['City'])
df['ordinal_City'] = factorised_City[0]

df[['ordinal_City', 'City']]

,ordinal_City,City
0,0,"Dallas, TX"
1,1,"Boston, MA"
2,2,"Los Angeles, CA"
3,2,"Los Angeles, CA"
4,2,"Los Angeles, CA"
...,...,...
185945,2,"Los Angeles, CA"
185946,3,"San Francisco, CA"
185947,3,"San Francisco, CA"
185948,3,"San Francisco, CA"


#### State

In [165]:
factorised_State = pd.factorize(df['State'])
df['ordinal_State'] = factorised_State[0]

df[['ordinal_State', 'State']]

,ordinal_State,State
0,0,TX
1,1,MA
2,2,CA
3,2,CA
4,2,CA
...,...,...
185945,2,CA
185946,2,CA
185947,2,CA
185948,2,CA


## Data Exploration

In [166]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 185950 entries, 0 to 185949
Data columns (total 19 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   Order ID           185950 non-null  int64         
 1   Product            185950 non-null  object        
 2   Quantity Ordered   185950 non-null  int64         
 3   Price Each         185950 non-null  float64       
 4   Order Date         185950 non-null  datetime64[ns]
 5   Line Total         185950 non-null  float64       
 6   At_Quarter_Change  185950 non-null  bool          
 7   Quarter            185950 non-null  int32         
 8   At_Month_Change    185950 non-null  bool          
 9   ordinal_Month      185950 non-null  int32         
 10  Month              185950 non-null  object        
 11  ordinal_Day        185950 non-null  int32         
 12  Day                185950 non-null  object        
 13  Week_Of_Month      185950 non-null  int64   

In [167]:
correlation = df.drop([
    'Order ID',
    'Quantity Ordered',
    'Product', 
    'Price Each',
    'Line Total',
    'Month', 
    'Day', 
    'City', 
    'State'
    ], axis=1).corr()
correlation

,Order Date,At_Quarter_Change,Quarter,At_Month_Change,ordinal_Month,ordinal_Day,Week_Of_Month,ordinal_Product,ordinal_City,ordinal_State
Order Date,1.000000,0.005298,0.967645,0.001865,0.995648,0.097060,0.186594,-0.003069,-0.000482,-0.000243
At_Quarter_Change,0.005298,1.000000,0.001956,0.571416,0.001267,0.000679,0.018590,-0.001684,-0.002950,-0.003614
Quarter,0.967645,0.001956,1.000000,0.000558,0.971585,0.017388,0.062606,-0.003312,-0.000630,0.000032
At_Month_Change,0.001865,0.571416,0.000558,1.000000,-0.000332,-0.002436,-0.005454,-0.002229,-0.001330,0.000008
ordinal_Month,0.995648,0.001267,0.971585,-0.000332,1.000000,0.016394,0.110052,-0.003091,-0.000689,-0.000399
ordinal_Day,0.097060,0.000679,0.017388,-0.002436,0.016394,1.000000,0.949634,-0.000856,0.001294,0.000899
Week_Of_Month,0.186594,0.018590,0.062606,-0.005454,0.110052,0.949634,1.000000,-0.000644,0.001110,0.000295
ordinal_Product,-0.003069,-0.001684,-0.003312,-0.002229,-0.003091,-0.000856,-0.000644,1.000000,-0.000725,0.000028
ordinal_City,-0.000482,-0.002950,-0.000630,-0.001330,-0.000689,0.001294,0.001110,-0.000725,1.000000,0.666425
ordinal_State,-0.000243,-0.003614,0.000032,0.000008,-0.000399,0.000899,0.000295,0.000028,0.666425,1.000000
